In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
import pandas as pd
import os
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot, plot
import math
from copy import deepcopy
import numpy as np
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, HBox, VBox
import random



In [ ]:
def frequency_to_note(frequency):
    # define constants that control the algorithm
    NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] # these are the 12 notes in each octave
    OCTAVE_MULTIPLIER = 2 # going up an octave multiplies by 2
    KNOWN_NOTE_NAME, KNOWN_NOTE_OCTAVE, KNOWN_NOTE_FREQUENCY = ('A', 4, 440) # A4 = 440 Hz

    # calculate the distance to the known note
    # since notes are spread evenly, going up a note will multiply by a constant
    # so we can use log to know how many times a frequency was multiplied to get from the known note to our note
    # this will give a positive integer value for notes higher than the known note, and a negative value for notes lower than it (and zero for the same note)
    note_multiplier = OCTAVE_MULTIPLIER**(1/len(NOTES))
    frequency_relative_to_known_note = frequency / KNOWN_NOTE_FREQUENCY
    distance_from_known_note = math.log(frequency_relative_to_known_note, note_multiplier)

    # round to make up for floating point inaccuracies
    distance_from_known_note = round(distance_from_known_note)

    # using the distance in notes and the octave and name of the known note,
    # we can calculate the octave and name of our note
    # NOTE: the "absolute index" doesn't have any actual meaning, since it doesn't care what its zero point is. it is just useful for calculation
    known_note_index_in_octave = NOTES.index(KNOWN_NOTE_NAME)
    known_note_absolute_index = KNOWN_NOTE_OCTAVE * len(NOTES) + known_note_index_in_octave
    note_absolute_index = known_note_absolute_index + distance_from_known_note
    note_octave, note_index_in_octave = note_absolute_index // len(NOTES), note_absolute_index % len(NOTES)
    note_name = NOTES[note_index_in_octave]
    return note_name

In [ ]:
collections = {"sm":"Scherbaum Mshavanadze",
               "guria":"Teach Yourself Gurian Songs",
               "megrelia":"Teach Yourself Megrelian Songs"}

collection_directories = {"sm":
                          ["GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919",
                           "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713",
                           "GVM019_DaleKojas_DidgoriVillage_Didgori_20160707",
                           "GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819",
                           "GVM097_KristeAghsdga_LakhushdiVillage_MuradGigoGivi_20160819"],
                          "guria":
                          ["Adila-Alipasha",
                           "Indi-Mindi",
                           'Mival Guriashi (1)' ,
                           'Pikris Simghera',
                           "Alaverdi",
                           "K'alos Khelkhvavi",
                           'Mival Guriashi (2)' , 
                           "Sabodisho",
                           "Khasanbegura",     
                           "Mok'le Mravalzhamieri",
                           'Sadats Vshobilvar',
                           "Beri Ak'vans Epareba", 
                           "Lat'aris Simghera",    
                           "Mts'vanesa Da Ukudosa", 
                           "Shermanduli",
                           "Brevalo",             
                           "Manana",         
                           'Nanina (1)',      
                           "Shvidk'atsa",
                           "Chven-Mshvidoba",    
                           "Maq'ruli",               
                           'Nanina (2)',          
                           'Supris Khelkhvavi',
                           'Didi Khnidan',     
                           "Masp'indzelsa Mkhiarulsa", 
                           "Orira",                
                           "Ts'amok'ruli",
                           "Gakhsovs, T'urpa",
                           "Me-Rustveli",        
                           "P'at'ara Saq'varelo"]}

directories = collection_directories["sm"]

data_dir = "/Akamai/Voice/data/pitches-vuv-new/"
working_coll = collections["sm"]
larynx = True
working_song = "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713"

algos = ['boersma', 'noll', 'crepe', 'maddox', 'hermes', 'yin'] # Note: later add praat
data3 = {}
locations = {}

for algo in algos:
    data3[algo] = {}
    locations[algo] = {}
    for direct in directories:
        data3[algo][direct] = {
            "mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        locations[algo][direct] = {
            "mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        


def separate(adir, algo):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

def load_songs():
    global data_dir, working_coll, working_song
    for algorithm in sorted(os.listdir(data_dir)):
        if algorithm in data3:
            print(algorithm)
            for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
                if collection != working_coll:
                    continue
                print(" ", collection)
                sm = (collection == "Scherbaum Mshavanadze")
                suffix = 'ALRX' if larynx and sm else 'AHDS' 
                for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                    if song != working_song:
                        continue
                    print("  ", song)
                    for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                        if (location[-4:] == '.txt'):
                            x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}", algo=algorithm)

                            freq = np.array(y).flatten()
                            i = 0
                            while i < len(freq):
                                sumFreq = freq[i]
                                a = 1
                                while i+a < len(freq) and freq[i+a] <= freq[i]*1.07 and freq[i+a] >= freq[i]*0.94:
                                    sumFreq += freq[i+a]
                                    a += 1

                                if a > 6 or i+a == len(freq)-1:
                                    for q in range(i,i+a+1):
                                        if q < len(freq):
                                            freq[q] = sumFreq/a
                                    i += a
                                else:
                                    freq[i] = 0
                                    i += 1  

                            notes = []
                            for i in range(0,len(freq)):
                                if freq[i] > 0:
                                    notes.append(frequency_to_note(freq[i]))
                                else:
                                    notes.append('N/A')

                            if not suffix in location:
                                data3[algorithm][song]['mix'] = (x, y, freq)
                                locations[algorithm][song]['mix'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                            else:
                                name_tag = location[(location.index(suffix) + 3):(location.index(suffix) + 6)]

                                if (name_tag[1] == '1' and not sm) or (name_tag[1] == '3' and sm):
                                    data3[algorithm][song]['bass'] = (x, y, freq)
                                    locations[algorithm][song]['bass'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"

                                elif name_tag[1] == '2':
                                    data3[algorithm][song]['middle'] = (x, y, freq)
                                    locations[algorithm][song]['middle'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                                else:
                                    data3[algorithm][song]['top'] = (x, y, freq)
                                    locations[algorithm][song]['top'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"

    print("\nLoaded song data from files into dictionary")
                       
load_songs()                                
                                

In [ ]:
selectedPoints = {}
targetSong = ""
traceIds = {}
targetAlgorithm = ""
undoState = []
   
def selection_fn(trace,points,selector):
    global targetSong, selectedPoints
    splitArray = points.trace_name.split(": ")
    algoName = splitArray[0]
    voice = splitArray[1]
    if(len(trace.selectedpoints) != 0):                
        selectedPoints[algoName+"-"+voice] = trace.selectedpoints
    ##print(points)
    #print(trace.selectedpoints)
    
def graph(song, targetAlgo, reload=False, type1 = 1):
    global targetSong, targetAlgorithm, undoState
    targetSong = song
    targetAlgorithm = targetAlgo
    print("target algo updated: " + targetAlgorithm)
    if reload:
        load_songs()
    print("plotting data from dictionary")
    traces = []
    type_list = []
    traceId = 0
    for algo, collection in data3.items():
        for name, part in collection.items():
            for audio_type, res in part.items():
                if name != song:
                    continue

                try:
                    trace = go.Scattergl(
                                x = res[0],
                                y = res[type1],
                                name=f"{algo}: {audio_type}",
                                mode="markers",
                                visible= (True if audio_type == "bass" else False),
                                selected=dict(marker=dict(color=DEFAULT_PLOTLY_COLORS[0]))
                            )

                    traceIds[algo+"-"+audio_type] = traceId
                    traceId += 1
                    traces.append(trace)
                    type_list.append(audio_type)
                except:
                    print(f"{algo}: {audio_type} not available")

    layout = go.Layout(title='Activity Heatmap')

    figure = go.Figure(data=traces, layout=layout)

    fig = go.FigureWidget(figure)

    for i in range(0,len(traces)):
        fig.data[i].on_selection(selection_fn)

    buttons = []
    labels = ['bass', 'middle', 'top', 'mix']
    for i, label in enumerate(labels):
        visibility = [label==current_type for current_type in type_list]
        button = dict(
            label = label,
            method = "update",
            args = [{ 'visible': visibility}]
        )

        buttons.append(button)

    updatemenus = list([
        dict(active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x = 0.005,
            y = 1.06,
        )
    ])

    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )


    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )


    fig['layout']['title'] = "Fixing " + targetAlgorithm + " for " + song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True
    fig['layout']['updatemenus'] = updatemenus
    
    undoState = copy.deepcopy(fig.data)


    def update_axes(Octave):
        print("Correct octave updated: " + Octave)
        
        
    def on_button_octave(x):
        global targetAlgorithm, targetSong
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    selectedpoints = list(selectedPoints[algo])
                    if(len(selectedPoints[algo]) != 0):    
                        i = selectedPoints[algo][0]
                        #print(data3[algoName][targetSong][voice][2])
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i-1
                        i = selectedPoints[algo][-1]
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i+1
                    selectedPoints[algo] = tuple(selectedpoints)
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData
                
    def on_button_octave_selected(x):
        global targetAlgorithm, targetSong
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData

    def on_button_delete(x):
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                for point in selectedPoints[algo]:
                    newData[point] = 0
            fig.data[traceIds[algo]].y = newData
    
    def on_button_delete_note(x):
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                selectedpoints = list(selectedPoints[algo])  
                i = selectedPoints[algo][0]
                print(data3[algoName][targetSong][voice][2])
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i-1
                i = selectedPoints[algo][-1]
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i+1
                selectedPoints[algo] = tuple(selectedpoints)
                for point in selectedPoints[algo]:
                    newData[point] = 0
                fig.data[traceIds[algo]].y = newData
            
    def on_button_preprocess(x):        
        global targetSong, targetAlgorithm
        undoState = copy.deepcopy(fig.data)
        print("Crepe post-process starting...")
        #fig.restyle(fig, 'selectedpoints', null)
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == "crepe":                
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                for i in range(0, len(fig.data[traceIds[algoName+"-"+voice]].y)):
                    if data3[algoName][targetSong][voice][2][i] == 0 and data3[algoName][targetSong][voice][1][i] != 0:
                        newData[i] = -1                
                fig.data[traceIds[algoName+"-"+voice]].y = newData
        print("Crepe post-process done.")
    
    def on_button_save(x):
        global targetSong
        print("Beginning saving files...")
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            #location = location.replace("pitches-vuv", "pitches-vuv-new")
            
            if algoName == "crepe":
                file = open(location, "w")
                for i in range(0,len(fig.data[traceIds[algoName+"-"+voice]].y)):
                    file.write(str(fig.data[traceIds[algoName+"-"+voice]].x[i])+" "+str(fig.data[traceIds[algoName+"-"+voice]].y[i]))
                    file.write("\n")
                file.close()
                print("Saved", location)
            
    def on_button_undo(x):
        global undoState
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == "crepe":                
                newData = np.array(undoState[traceIds[algoName+"-"+voice]].y)          
                fig.data[traceIds[algoName+"-"+voice]].y = newData
    
    def on_button_deselect(x):
        global undoState
        for scatter in fig.data:                   
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = None
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = []

    button_preprocess = Button(description="Post-process Crepe", button_style='danger')
    button_octave = Button(description="Correct note", button_style='danger')
    button_octave_selected = Button(description="Correct selected", button_style='danger')
    button_delete = Button(description="Delete selected", button_style='danger')
    button_delete_note = Button(description="Delete note", button_style='danger')
    button_save = Button(description="Save Crepe", button_style='danger')
    button_undo = Button(description="Undo", button_style='danger')
    button_deselect = Button(description="Deselect", button_style='danger')
    button_preprocess.on_click(on_button_preprocess)
    button_octave.on_click(on_button_octave)
    button_octave_selected.on_click(on_button_octave_selected)
    button_delete.on_click(on_button_delete)
    button_delete_note.on_click(on_button_delete_note)
    button_save.on_click(on_button_save)
    button_undo.on_click(on_button_undo)
    button_deselect.on_click(on_button_deselect)

    #button_next.observe(on_button_next, )

    display(widgets.VBox([fig,HBox([button_preprocess, button_delete, button_delete_note, button_octave_selected, button_octave, button_save, button_undo, button_deselect])]))
    

### Can Correct

- Crepe (target algo) notes that should be 0 (usually another voice is singing)
- Crepe (target algo) notes for which another algorithm has a better estimate (in another octave)

### Cannot Correct

- consonants that show up as 0's or other notes (usually no algorithm has a good estimate)
- notes in the wrong octave but for which no algorithm has a good estimate in the right octave
- notes in the right octave that sound wrong 



In [ ]:
graph(working_song,'crepe', False)

#display(VBox([button_delete,button_next]))

## Problem sections

### Problems:

- partly-corrected notes
- 0's (probably from Voiced-Unvoiced algorithm) that should be notes

### Non-problems:

- consonants that show up as 0's or other notes
- notes that sound wrong but for which no algorithm offers a correction

#### Syntax

- start_time, algorithm, problem
- first_start_time-last_start_time, algorithm, problem (if there are 2 or more sections in succession)

### Scherbaum Mshavanadze songs

#### GVM009 - Bat'onebis Nanina - Larynx

Bass 

Middle - perfect

- places I was right to delete: 20.4, 46.5, 54.7, 62.1, 77.0, 91.8
- places I was wrong, maybe
  - 68.6, 69.9, but I think the notes I deleted were lower than the middle voice dips
  - 83.4, but I think the note I deleted was lower than the middle voice dip

Top - perfect 
- all the "middle notes" I deleted were really part of the top voice

#### GVM009 - Bat'onebis Nanina - Headset

Bass 

Middle

- 20.4, crepe deleted top note 3rd up on 'tkb'
- 46.5, crepe deleted top note 3rd up on 's'
- 54.7, crepe deleted top note 3rd up on 't'
- 62.1, crepe deleted top note 3rd up on 'sh'
- 68.6, 69.9, crepe deleted bottom notes on 'd', 't'
- 77.0, crepe deleted top note 3rd up on 'ts'
- 83.4, crepe deleted bottom note on 'd'
- 91.8, crepe deleted bottom note on 'tqv'
- 118.3, crepe false 0, v-uv

Top - Yin is not registered, a little to the right, stopped deleting consonants except in obvious cases

- 13.9, 14.1, crepe deleted middle note 3rd down on 'v', 's'
- 23.6, crepe deleted middle note 3rd down on 'd'
- 30.4-32.7, crepe deleted middle notes 3rd down on 'b', 'd'
- 37.4-39.5, crepe did not delete noodles (middle notes?)
- 84.0, crepe deleted middle note 3rd down on 'b'
- 91.7, crepe deleted middle note 3rd down on 'tqv'

#### GVM017 - Chven Mshvidoba - Larynx

Bass - deleted middle notes in most silent parts

Middle - this mic picked up other voices

- deleted parts at 50Hz that should be 0
- 28.5, crepe deleted section of silence with non-obvious start and end times
- 41.5-42, corrected octave with Maddox
- 42.6-43.1, corrected octave with Maddox
- 43.7, crepe deleted section of silence with non-obvious start and end times
- 53.5, crepe deleted section of silence with non-obvious start and end times
- 56.3, crepe deleted section of silence with non-obvious start and end times
- 70.8, crepe deleted section of silence with non-obvious start and end times
- 78.0, crepe deleted section of silence with non-obvious start and end times
- 83.5, crepe deleted section of silence with non-obvious start and end times
- 89.4, crepe deleted section of silence with non-obvious start and end times
- 96.2, crepe deleted section of silence with non-obvious start and end times
- 101.6, crepe deleted section of silence with non-obvious start and end times
- 108.1-109.4, corrected octave with Noll
- 109.6, crepe deleted section of silence with non-obvious start and end times
- 127.5, crepe deleted section of silence with non-obvious start and end times
- 133.6, crepe deleted section of silence with non-obvious start and end times
- 140.7, crepe deleted section of silence with non-obvious start and end times
- 143.4, crepe deleted section of silence with non-obvious start and end times
- 150.6, crepe deleted section of silence with non-obvious start and end times
- 154.7, crepe deleted section of silence with non-obvious start and end times
- 157.7, crepe deleted section of silence with non-obvious start and end times
- 161.0, crepe deleted section of silence with non-obvious start and end times
- 165.6, crepe deleted section of silence with non-obvious start and end times

Top - almost perfect

- 40.4-40.9, corrected octave with Noll
- 166.2-166.6, corrected octave with Noll

#### GVM017 - Chven Mshvidoba - Headset - may have fixed some octaves, I forget; doing Larynx

Bass (Track 3)

Middle

Top (Track 1)

#### GVM019 - Dale Kojas - Larynx 

Bass 

Middle 

- 23.4-23.85, corrected octave with Yin, Maddox
- 30.7-31.2, corrected octave with Maddox
- 34.4-35.2, corrected octave with Maddox

Top - mostly false 0's, from v-uv?

#### GVM019 - Dale Kojas - Headset

Bass (Track 3)

- removed all 'sh' consonants that are long and have high notes

Middle - top is too loud, need boxes

Top (Track 1)

- removed a few long 'sh' consonants

#### GVM031 - Elia Lrde - Larynx - no need for headset

Bass - a few middle notes (below...?) deleted during silence

Middle - perfect

Top - perfect

#### GVM097 - Kriste Aghsdga - Larynx

Bass - mostly false 0's, from v-uv?

Middle - perfect

Top - perfect

#### GVM097 - Kriste Aghsdga - Headset

Bass - deleted middle notes during silence

Middle - also perfect

Top - also perfect


### Gurian songs

#### Alaverdi

Bass

Middle - crosses top a lot and is in 4-5 with top a lot

- 4.0, crepe deleted top note on 's'
- 5.2, crepe deleted top note 2nd up on 't'
- 10.3, crepe deleted top note 2nd up on 'ns'
- 31.8, crepe deleted top note 3rd up on 'chkh'
- 73.5, crepe deleted top note 3rd up on 't'
- 91.0, 92.1, crepe did not delete noodles (top notes 3rd up?)
- 103.1, crepe deleted top note 3rd up on 'l'

Top

- 2.9, crepe deleted middle note 3rd up on 'j'
- 75.6, crepe deleted middle note (crossing) on 'l'
- 93.8, crepe deleted middle note (crossing) on 'rd'
- 103.2, crepe deleted middle note 3rd down on 'd'
- 111.8, 112.5, crepe deleted middle note 3rd down on 'd'
- 119.8, crepe false 0, from v-uv


#### Beri Ak'vans Epareba

Bass

- 20.4, crepe deleted middle note 3rd up on 'kv' (?, didn't delete next noodles)
- 25.1, crepe deleted middle/top note on 'd' (?)
- 65.3, 66.0, crepe did not delete noodles (middle notes?) on 'l', 'd'
- 97.1, crepe deleted middle note on 'mtv'
- 102.0, crepe deleted middle note on 'd'
- 113.0, crepe deleted middle note on 'd'

Middle - crosses top a lot

- 20.6-20.9, crepe deleted bass notes 3rd down on consonants
- 22.6, crepe deleted bass note 3rd down on consonant
- 23.0, crepe deleted top note 2nd? up on consonant
- 26.2, crepe deleted top note 3rd up on 'd'
- 58.9-60.0, crepe deleted top notes (below) on consonants
- 61.7, 62.1, crepe deleted top notes (crossing) on consonants
- 64.9, crepe deleted top note 3rd up on 'n'
- 67.7, crepe deleted top note 3rd up on 'd'
- 77.1-78.3, crepe false 0 notes, v-uv
- 96.0-98.0, crepe deleted top notes (crossing) on consonants
- 101.9, crepe deleted top note 3rd up on 'n'

Top

- 8.8, crepe deleted middle note on 'd'
- 15.2, crepe deleted middle note on 'd'
- 20.3, crepe deleted bottom note on 'z'
- 54.6, crepe deleted middle note on 'd'
- 58.4, 59.3, crepe deleted false high notes on 's', 'ch'
- 85.9, crepe deleted bottom note on 'd'
- 95.7, 96.9, 97.5, crepe deleted bottom notes on 's'
- 106.0, crepe deleted bottom note on 'd'
- 110.6, 111.2, crepe deleted middle notes on 'd', 'v'


#### Brevalo

Bass

Middle

Top

#### Chven-Mshvidoba

Bass

Middle 

- 119.7, 122.8, crepe deleted bass notes on consonants

Top - close to middle and crossing a lot

- 124.3, crepe deleted middle note 2nd up on 'qv'
- 137.4, crepe deleted middle note unison on 'd'
- 154.0-155.4, crepe deleted middle notes on consonants
- 202.7, crepe deleted middle note 2nd down on 'd'

#### Didi Khnidan

No bass part

#### Gakhsovs, T'urpa

Bass

Middle

- 18.5, crepe deleted top note 3rd up on 'd'
- 32.8, crepe deleted top note 3rd up on 'd'
- 35.5, crepe deleted bottom note 3rd down on 'p'
- 38.5, 41.1, crepe deleted top notes 3rd up on 'd'
- 72.9, crepe deleted top note 2nd up on 'sh'
- 73.9, crepe deleted bottom note 4th down on 'sh'
- 75.8, crepe deleted top note 3rd up on 'd'
- 87.0-88.4, crepe false 0 notes, v-uv

Top - close to middle, hard to tells scoops from false notes

- 14.0, crepe deleted middle note on 'sh'
- 16.3, crepe deleted bottom note on 'rt'
- 42.7, 43.0, crepe deleted middle notes on consonants
- 44.1, crepe deleted middle note on 's'
- 45.2, crepe deleted bottom note on 'kv'
- 73.4-81.8, crepe deleted middle notes on consonants

#### Indi-Mindi

Bass

Middle

- 59.5, crepe false note 5th up (top), some algos partly correct
- 66.9-67.0, crepe deleted bottom note on 'kh'
- 87.3, crepe deleted bottom note on 'ls'
- 93.5, crepe deleted bottom note on 'd'?
- 106.4-107.9, crepe deleted bottom notes on consonants, mostly 's'

Top

- 46.6, crepe deleted middle note on 'sh'
- 66.5, crepe deleted middle note on 'kh'
- 85.9-88.1, crepe deleted middle notes on consonants
- 100.5, crepe 6th high, no algo correct, looks like 'm'
- 105.4-107.6, crepe deleted middle notes on consonants
- 110, crepe false 0 on solo 'oi', v-uv

#### K'alos Khelkhvavi

Bass

Middle - close to top on verses

- 12.26, crepe deleted bass note 4th down on 's'
- 33.9-34.2, crepe false 0 notes from v-uv
- 56.2-58.0, crepe deleted top notes on consonants (mistaken on 'd', 'b'?)
- 64.0-69.4, crepe deleted bottom notes on consonants
- 72.2, 72.6, crepe didn't delete apparent top notes on 'd', 'b'
- 95.0-99.5, crepe deleted bottom notes on consonants

Top

- 19.8, 20.2, crepe deleted middle notes on 'v', 'h' (tough call)
- 24.5, 25.9, crepe deleted middle notes on 'd'
- 54.4-55.9, 59.6, crepe didn't delete apparent middle notes on consonants
- 64.1, 64.6, crepe deleted middle notes 3rd up on consonants
- 70.4, crepe deleted middle note 3rd up on 's'
- 82.2, crepe deleted middle note 3rd down on 'd'
- 84.8-86.5, crepe didn't delete apparent middle notes on consonants 

#### Khasanbegura

Bass

- 26.2, crepe deleted middle note 4th up on 'd'
- 42.4, crepe deleted middle note 3rd up on 'v' (?)
- 47.6-48.4, crepe choir overlap, bass solo 4th up
- 77.7, crepe deleted middle note 3rd up on 'd'
- 85.6, crepe choir false note 9th up, no correct algo
- 87.6-88.5, crepe choir overlap, bass solo various intervals
- 125.9, crepe choir false note 9th up, no correct algo
- 166.0-168.0, crepe choir false notes, no correct algo
- 168.0-169.9, crepe choir overlap, bass solo various intervals
- 180.9, crepe deleted break after indistinct note, no correct algo

Middle

- 8.5, 9.4, crepe false top/bass notes, Noll is correct
- 26.8, 27.2, crepe choir false bass notes, no correct algo
- 28.2, 28.6, crepe choir false bass note, Noll mostly correct but some parts uncorrected
- 29.3, 30.6, 31.7, crepe choir false bass notes, some algos correct on some parts
- 38.7, 39.9, 40.6, 41.5, 42.0, 42.3, 42.5, 42.8, 43.2, crepe choir false notes, mostly bass, no algos correct
- 47.8-56.6, crepe solo deleted top/bass notes on consonants
- 67.6-87.1, crepe choir confused with bass on many notes
- 108.7-127.5, crepe choir confused with bass on many notes
- 119.3, crepe choir deleted top note on 's'
- 129.4-138.9, crepe solo deleted top/bass notes on consonants
- 148.5-164.6, crepe choir confused with bass on many notes
- 167.7-188.1, crepe solo deleted top/bass notes on consonants
- 188.1-211.2, crepe choir confused with bass on many notes

Top - crepe is in the wrong octave a lot, regular and krimanchuli, Boersma correct 
 
- 15.2, 16.9 - crepe 4th up, Boersma correct
- 20.5, crepe wrong octave, no correct algo, tried to correct with Yin, made it worse
- 25.3-26.3, crepe wrong notes from choir 
- 62.1, crepe 1 octave high, only partly corrected by Hermes, Yin
- 67.4-69.5, crepe mostly 5th low, partly 4th high (bass choir), Boersma and Hermes mostly correct
- 108.2-109.1, crepe 1 octave high, parts uncorrected by Hermes, Yin
- 148.3-150.4, crepe mostly 5th low (bass choir), Boersma and Hermes mostly correct
- 188.0-189.9, crepe mostly 5th low (bass choir), Boersma and Hermes mostly correct
- 210.9-214.2, crepe mostly 5th low (bass choir), Boersma and Hermes mostly correct


#### Lat'aris Simghera

Bass

- 52.9, crepe deleted middle note on 'sh'
- 153.1, crepe deleted middle note on 's'
- 170.9, crepe deleted middle note on 's'
- 203.4, 203.9, crepe middle note 5th up, Noll is correct

Middle

- 84.8, crepe deleted bass note on 'sh'
- 104.1, crepe deleted bass note on 's'
- 131.3, crepe deleted bass note on 'kh'
- 148.7, crepe deleted top note on 'ch'
- 151.8, crepe deleted bass note on 'kh'
- 171.0, crepe deleted bass note on 'sh'
- 192.8, crepe deleted top note on 'ts'

Top

- 18.8, crepe deleted middle note 1st down on 'd'
- 20.5, crepe deleted middle? note on 's' (maybe the next note on 't' should be deleted, too?)
- 53.0, 53.3 deleted middle note 1st down on 'stv'
- 54.4, crepe deleted middle note? (unison?) on 's'
- 84.6, 84.7, crepe deleted middle note? on 'shv'
- 92.9-94.0, crepe false middle notes, Noll correct an octave down
- 101.8, crepe false middle note, Noll correct
- 103.9, crepe deleted middle note? (unison?) on 's'
- 170.8 deleted middle note 1st down on 's'
- 172.4-173.2 mistakenly deleted middle notes 1st down, Noll correct
- 177.5, crepe deleted middle/bass note 1st down on 'd'
- 197.0, crepe deleted middle/bass note 1st down on 'dm'
- 215.2, crepe deleted middle/bass note? (unison, 1st up) on 's'
- 217.8, crepe deleted middle note 1st down on 'l'
- 220.4, crepe deleted middle note 1st down on 'vl'?

#### Manana

Bass

- 106.0, crepe deleted middle note during 'kh'

Middle

- 9.9, crepe deleted top/bottom note during 's'
- 14.7, crepe false top note 4th up
- 16.2, 16.8, crepe false bottom note 3rd down
- 17.9, 19.1, crepe deleted bass notes during 's', 'kh'
- 23.1, crepe deleted top note during 't'
- 38.2, crepe deleted top note during 't'
- 45.4, crepe deleted top note during 's'?
- 47.5-47.8, crepe deleted top note during 'sh'
- 54.1, crepe deleted top note during 'l'
- 70.0, crepe deleted bottom note during 's'
- 74.1, crepe deleted bottom note during 'kh'
- 77.0, crepe deleted bottom note during 'sh'
- 80.9, crepe deleted top note during 'b'
- 94.9, crepe deleted top note during 'kh'
- 104.6, crepe deleted bottom note during 'sh'
- 109.3, 109.8, crepe deleted bottom, top notes during 't', 'k'

Top

- 24.5, crepe deleted middle note during 'kh'
- 47.3, crepe deleted middle note during 'msh'
- 69.8, crepe deleted middle note during 's'
- 71.1, crepe deleted middle note during 's'
- 76.7, crepe deleted middle note during 'sh'
- 94.5, crepe deleted middle note during 'kh'
- 109.0, crepe deleted middle note during 't'
- 116.8, crepe false 0 from v-uv

#### Maq'ruli

Bass

- 31.1, crepe false middle note, Noll correct
- 83.5, crepe false middle note, Boersma correct

Middle - hard to hear

- 15.6, crepe deleted bass note during 'ch'
- 19.1?, crepe mistake? sounds like middle sings bass note for a moment
- 21.9-22.9, crepe, various wrong notes, Noll mostly correct
- 24.6, crepe false bass note, Noll correct
- 45.5-46.6, crepe, various wrong notes, Noll mostly correct
- 86.0, crepe false bass note, Noll correct
- 92.4, crepe false bass note, Noll correct
- 101.2-106.3, crepe, various wrong notes, Noll mostly correct
- 114.9-116.8, crepe, various wrong notes, Noll mostly correct
- 126.9-127.3, crepe, various wrong notes, Noll mostly correct
- 144.0-148.4, crepe, various wrong notes, Noll, Boersma, Hermes have some corrections

Top - might have called a lot of n's wrong notes

- 13.1, 13.3, crepe middle notes, Noll correct
- 20.7, crepe middle note, all voices wrong
- 25.0-26.4, crepe middle notes, Noll mostly correct
- 41.2-42.7, crepe wrong octave, no correct algo (some corrections on some notes)
- 49.7-50.0, crepe middle/bass note, other algos correct on parts of note
- 72.4-73.4, crepe various wrong notes, Noll, Boersma, Hermes have most of corrections
- 73.5, crepe old choir notes, misses new krimanchuli, Yin partly correct
- 95.6-96.8, crepe various wrong notes, Noll mostly correct
- 107.1, crepe middle note, Noll correct
- 112.2, crepe deleted middle note during 'n'
- 118.3, crepe middle note, Noll correct
- 122.8, crepe middle note, no correct algo
- 143.9, crepe middle note, Noll partly correct
- 149.5, crepe middle note, other algos have parts of correct note

#### Masp'indzelsa Mkhiarulsa

Bass

- 54.6-56.4, crepe deleted middle/top notes during 'ts, 'm', 's'
- 77.8, crepe deleted middle note during 's'
- 105.5-107, crepe false 0 estimate, v-uv

Middle 

-76.3-76.6, crepe deleted bass note during 'sh'

Top - crosses or is under middle a lot

- 19.8, crepe deleted top note during 'v'
- 81.9, crepe deleted top note during 'd'


#### Me Rustveli

Bass

- 46.8, crepe deleted middle note during 's'
- 49.2, crepe deleted middle note during 's'
- 84.4, 86.0 crepe deleted middle notes during 'ts', 's'
- 116.7, 117.8 118.3 crepe deleted middle notes during 'v', 'ch', 's'
- 151.2, crepe deleted middle note during 'd'
- 171.7, crepe false 0 estimate, boersma is correct
- 181.5-181.6, crepe deleted middle note during 'ch'?
- 185.3, crepe deleted middle note during 'kv'?

Middle

- 17.6, crepe deleted top note during 's'
- 51.1, crepe deleted bass note during 'ch'?
- 119.4, crepe deleted bass note during 'ts'

Top - crosses middle a lot, and takes the 4 in 1-4-5 chords

- 15.9-16.1, crepe deleted middle notes during consonants
- 83.2-84.1, crepe deleted middle notes during consonants
- 148.1-150.9, crepe deleted middle notes during consonants
- 167.8, 168.7, crepe false high note estimate, noll is correct
- 194.0-194.3, all algos confuse top and middle
- 197.6-197.8, crepe false 0 estimate, v-uv mistake

#### Mival Guriashi (1)

Bass

- 14.0, crepe deleted top note during consonant
- 50.4, crepe deleted middle note during consonant
- 64.5, crepe deleted middle note during consonant
- 80.3-82.1, crepe deleted middle/top notes during consonants
- 96.3-97.7, crepe deleted middle notes during consonants

Middle

- 13.9, crepe deleted top note during 'sh'
- 49.6, crepe didn't delete 'd', sounds like top
- 64.6, crepe deleted top note during 'kvshm'
- 81.9, crepe deleted top note during 's'
- 113.2, crepe deleted bass note during 'ch'
- 127.5, crepe deleted top note during 'd'

Top

- 13.8, crepe deleted middle note during 'sh'
- 32.1, 32.8 crepe deleted middle notes during 'ts', 'n'
- 64.5, crepe deleted middle note during consonant ('ghdm'?)
- 81.8, crepe deleted bass note during 's'
- 95.7, 95.9, 97.1 crepe deleted middle notes during 'm', 'kv', 's'
- 128.6, crepe deleted middle note during 'b'


#### Mival Guriashi (2)

Bass

Middle

- 35.7, crepe deleted bass note during consonant
- 39.0, crepe deleted top note during consonant
- 58.0-59.1, crepe deleted bass notes during consonants
- 105.5-108.9, crepe deleted bass notes during consonants
- 112.8, crepe deleted top note during consonant

Top

- 10.4, crepe deleted bass note during consonant
- 18.2, crepe deleted middle note aug-5th down
- 18.8-19.2, crepe delete bass note 8th down (no correct algos)
- 75.2-75.3, crepe delete bass note 8th down (no correct algos)
- 76.0-76.7, crepe delete middle notes 4th-5th down 
- 98.2-98.4, crepe delete bass note 8th down (no correct algos)
- 107.2, crepe delete bass note 4th down 


#### Mok'le Mravalzhamieri

Bass

Middle - crosses top a lot

Top

- 3.2, 4.0, 9.5, 10.1, 11.6, 17.4, 19.7 crepe estimates 4th or 5th too high (Noll mostly correct) 
- 18.7 crepe estimate 5th too high, miscorrected (Noll)

#### Mts'vanesa Da Ukudosa

Bass

- 43.9-45.9, crepe 0
- 87.8-88.4, crepe 0
- 129.4-129.8, crepe 0

Middle - corrected consonants when top voice present

Top - corrected consonants when bottom voice present

- 15.6, crepe bass note
- 29.0, 32.2, 39.5, 72.4, 82.4, 102.8, 110.8, 113.5, 123.1, 126.3 crepe deleted mystery estimates during apparent silence
- 70.4, crepe bass note
- 83.5, crepe bass note (top note one octave up, all algorithms pick the bass note)
- 102.1, crepe bass note

#### Nanina (1)

(Lost notes to git?)

#### Nanina (2)

Bass

Middle

Top

- 56.0, crepe estimate is another voice
- 66.7, crepe miscorrected short note to 0

#### Orira

Bass

Middle - hard to tell when the estimate is a bass note throughout; Noll better in some passages

Top - some estimates are other voices: 21.3, 46.0, 63.6, 77.7 (no better estimate)

- 87.3, crepe corrected other voice to a note above Boersma; didn't correct 87.8, 88.4

#### P'at'ara Saq'varelo

Bass

Middle - hard to tell when the estimate is a bass note throughout

- 68.4, crepe consonant changed to 0 because estimate is bass note

Top 

- 

#### Pikris Simghera

Bass

Middle

Top

#### Sabodisho

Bass

- 106.7-107.3, crepe 1 octave low

Middle

- 86.3, crepe 1 octave low
- 92.9-93.7 crepe 0
- 105.3, crepe 0
- 116.1-116.6, crepe 0
- 128.7-130.2, crepe 0
- 148.5-149.8, crepe 0
- 168.3-169.3, crepe 0
- 194.2-197.6, crepe 0

Top


#### Shermanduli

Bass

Middle

- 29.4, crepe deleted top note 3rd up on 'd'
- 55.6, 56.1, crepe deleted top notes 3rd up on 'n'

Top

- 25.0, crepe deleted middle note 4th down on 'l'


#### Shvidk'atsa

Bass

- 61.8-62.5, crepe choir false notes, all algos
- 91.6-92.5, crepe choir false notes, all algos

Middle

- 20.9-21.6, crepe (1st-2nd choir change) false 0, some algos partly correct
- 32.9-34.8, crepe deleted top/bass notes on consonants
- 37.2, 38.1, crepe did not delete low krimanchuli notes, hard to tell if the correspond to middle consonants
- 59.0, crepe deleted bass note on 'd'
- 59.6, crepe false bass note (above), Noll partly correct
- 67.3-69.3, did not delete notes 2nd down on consonants (bass way below)
- 82.2-82.8, crepe (1st-2nd choir change) false 0, Noll mostly, Hermes partly correct
- 89.4-89.7, crepe false 0, other algos mostly pick up bass above

Top

- 30.8-32.4, crepe (1st-2nd choir change) false notes, all algos partly correct


#### Supris Khelkhvavi

Bass

Middle

- 85.0, crepe deleted top note 5th up on 't'

Top

- 6.6, crepe false 0, Hermes and Yin partly correct
- 39.8, crepe false middle note (5th up), Noll correct
- 104.1, 105.3 crepe deleted middle note 5th down on 's'
- 129.3, crepe false 0, Noll correct on 'v'
- 139.3, crepe deleted middle note 5th? up on 'shl'
- 145.8, 146.6, crepe deleted middle notes on 'tsr', 'kh'


#### Ts'amok'ruli

Bass 

Middle

Top

- 73.7, crepe false middle note during short krimanchuli squawk, no algo correct


### Megrelian Songs